In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from textblob import TextBlob
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv


In [41]:
df=pd.read_excel("CANIS_PRC_state_media_on_social_media_platforms-2023-11-03.xlsx")

In [42]:
df=df[((df['Region of Focus']=='Anglosphere')|(df['Region of Focus']=='Canada'))]

In [43]:
handles=df['X (Twitter) handle'].values
accounts=[h for h in handles if isinstance(h,str) ]

In [44]:
accounts.remove("anosiwang")

In [39]:
if os.path.exists("data.json"):
    with open('data.json', 'r') as json_file:
        existing_data = json.load(json_file)
    existing_accounts=list(existing_data.keys())
    accounts=[elem for elem in accounts if elem not in list(existing_data.keys())]

In [13]:
def configure():
    load_dotenv()
configure()

In [23]:
def get_content(driver, username):
    driver.get(f'https://twitter.com/{username}')
    time.sleep(6)
    user_name=str(os.getenv('twitter_username'))
    password=str(os.getenv('twitter_password'))
    driver.find_element(By.TAG_NAME, 'input').send_keys(
            user_name
        )
    # Click next
    driver.find_element(By.XPATH, "//*[text()='Next']").click()
    # Input password
    time.sleep(2)
    driver.find_element(By.NAME, "password").send_keys(
        password
    )
    time.sleep(2)
    # Click login
    driver.find_element(By.XPATH,'//div[@data-testid="LoginForm_Login_Button"]').click()
    #login.find_element(By.XPATH, "//*[text()='Log in']").click()
    time.sleep(5)  # Let the page load
    body=WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.TAG_NAME,'body')))
    #body = driver.find_element(By.TAG_NAME,'body')
    time.sleep(2)
    section = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME,'section')))
    num_followers=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div/div/div/div[5]/div[2]/a/span[1]/span')))
    num_post_to_scrape=250
    i=0
    data=[{'post':""}]
    content_check=[]
    # Scroll until the desired number of followers is reached
    while i < num_post_to_scrape:
        body.send_keys(Keys.PAGE_DOWN)
        i+=1
        time.sleep(2)
        post=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//div[@data-testid="cellInnerDiv"]')))
        #post = section.find_element(By.XPATH,'//div[@data-testid="cellInnerDiv"]')
        content=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//div[@data-testid="tweetText"]')))
        #content=post.find_element(By.XPATH,'//div[@data-testid="tweetText"]')
        content_check.append(content.text.strip())
        time.sleep(2)
        #datetime=post.find_element(By.TAG_NAME,'time')
        datetime=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME,'time')))
        datetime=datetime.get_attribute('datetime')
        if (content.text.strip()!=data[-1]['post']):
            #likes=post.find_element(By.XPATH,'//div[@role="group"]')
            likes=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//div[@role="group"]')))
            engagement=likes.get_attribute('aria-label')
            eng_=engagement.split(",")
            eng_=list(map(lambda x:x.split(),eng_))
            if engagement:
                eng_={TextBlob(v).words.pluralize()[0] if not v.endswith('s') else v: int(k) for k,v in eng_}
            else:
                eng_={}
            for e in ['comments','likes','reposts','bookmarks','views']:
                if e not in eng_:
                    eng_[e]=0
            eng_['post']=content.text.strip()
            eng_['followers']=int(num_followers.text.replace(',', '').replace(".",'').replace("K",'00').replace('k','00'))
            eng_['date_time']=datetime
            data.append(eng_)
            time.sleep(1)  # Adjust sleep time based on your internet speed
        
    return data 

In [ ]:
all_data={}
# Set up the Selenium WebDriver
#driver = webdriver.Chrome()
for j,acc in enumerate(accounts):
    driver = webdriver.Chrome()

    # Replace 'target_account' with the username of the account you want to analyze
    target_account = acc
        
    # Get the number of followers for the target account
    target_engagement = get_content(driver, target_account)
    all_data[acc]=target_engagement[1:]
    if j%4==0:
        if os.path.exists("data.json"):
            with open('data.json', 'r') as json_file:
                existing_data = json.load(json_file)
            existing_data.update(all_data)
            with open("data.json", 'w') as json_file:
                json.dump(existing_data, json_file)
        else:
            with open("data.json", 'w') as json_file:
                json.dump(all_data, json_file)

In [8]:
with open('data.json', 'r') as json_file:
        existing_data = json.load(json_file)